In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


DEV, TEST and PROD deployments will be hold in different SCHEMAS

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
session.sql('create or replace schema DEV').collect()
session.sql('use schema DEV').collect()


In [ ]:
session.sql('use schema DEV').collect()


Here we are using the power of Cortex LLMs to create some feedback comments

In [ ]:
comments_sql= """
        create or replace table comments_temp (id number, comment VARCHAR) as 
        
        select 1, snowflake.cortex.complete ('mistral-large2', 'write comment complaining about the product SkiBoots123. They were broken after 2 days of usage. You are very dissatified')
        UNION
        select 2, snowflake.cortex.complete ('mistral-large2', 'write comment complaining about a defect in a recent purchase indicating you are not satisfied and will not buy in the shop again. Do not indicate any date, product or shop ')
        UNION
        select 3, snowflake.cortex.complete ('mistral-large2', 'write comment complaining about a recent shipment where the package was broken. Do not indicate any date, product or shop ')
        UNION
        select 4, snowflake.cortex.complete ('mistral-large2', 'write comment where you complain a litte bit about support not calling you back. Do not indicate any date, product or shop ')
        UNION
        select 5, snowflake.cortex.complete ('mistral-large2', 'write a neutral coment about a recent call you had with support. Do not indicate any date, product or shop ')
        UNION
        select 6, snowflake.cortex.complete ('mistral-large2', 'write a neutral comment about a recent purchase you have done. Do not indicate any date, product or shop ')
        UNION
        select 7, snowflake.cortex.complete ('mistral-large2', 'write a comment indicating you are satisfied with a recent purchase. Do not indicate any date, product or shop ')
        UNION
        select 8, snowflake.cortex.complete ('mistral-large2', 'write a comment indicating you are satisfied with a recent purchase and you will recommend the shop. Do not indicate any date, product or shop ')
        UNION
        select 9, snowflake.cortex.complete ('mistral-large2', 'write a comment indicating you are satisfied with a recent support received by a shop assistant. Do not indicate any date, product or shop ')

        """

comments_tb = session.sql(comments_sql).collect()

In [ ]:
select * from comments_temp;

In [ ]:
drop table if exists customers;
drop table if exists sales;
drop table if exists feedback_raw;
drop table if exists feedback_sentiment;

Function to add some fake data. This will generate customers, transactions and feedback that will be used later to understand customer purchase behaviours and detect when a customer is on high probability of not buying again so we can take some actions

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from snowflake.snowpark import types as T
from snowflake.snowpark import functions as F

num_customers = 10000
num_transactions = num_customers * 15
num_feedback_reports = num_customers * 2

locations = ['Madrid', 'Barcelona', 'Paris', 'London', 'Munich', 'Rome']
payment_methods = ['Credit Card', 'Paypal', 'Apple Pay']
customer_segments = ['Planitum', 'Gold', 'Silver', 'Standard']
product_segments = ['Fashion', 'Electronics', 'Beauty', 'Groceries', 'Home', 'Toys', 'Books']


def generate_data (first_transaction_date, period):

    customers = []
    for i in range(num_customers):
        customer_id = f"CUST-{i}"
        age = random.randint(18, 99)
        gender = 'Male' if random.choice([True, False]) else 'Female'
        location = random.choice(locations)
        signup_date = first_transaction_date - timedelta(days=random.randint(365,900))
        customer_segment = random.choice(customer_segments)
        
        customers.append ([customer_id, age, gender, location, signup_date, customer_segment])

    df_customers = pd.DataFrame(customers, columns= [
        "CUSTOMER_ID", "AGE", "GENDER", "LOCATION", "SIGNUP_DATE", "CUSTOMER_SEGMENT"
    ])

    customers = session.create_dataframe(df_customers).drop_duplicates()
    customers = customers.with_column("signup_date", F.col("signup_date").cast(T.DateType()))
    customers.write.mode("overwrite").save_as_table("CUSTOMERS")
    #customers.write.csv('@CSV_STAGE/customers.csv', overwrite=True, single=True)


    transactions = []
    for i in range(num_transactions):
        transaction_id = f"TRANS-{i+1}"
        customer_id = f"CUST-{random.randint(1, num_customers)}"
        transaction_date = first_transaction_date + timedelta(days=random.randint(0,period))
        total_amount = round(random.uniform(50, 5000), 2)
        num_items = random.randint(1, 10)
        discount_applied = bool(random.randint(0, 1))
        payment_method = random.choice(payment_methods)
        
        transactions.append([transaction_id, customer_id, transaction_date, total_amount, num_items, discount_applied, payment_method])

    # Creating the DataFrame for transactions
    df_transactions = pd.DataFrame(transactions, columns=[
        "TRANSACTION_ID", "CUSTOMER_ID", "TRANSACTION_DATE", "TOTAL_AMOUNT", 
        "NUM_ITEMS", "DISCOUNT_APPLIED", "PAYMENT_METHOD"
    ])

    transactions = session.create_dataframe(df_transactions).drop_duplicates()
    transactions = transactions.with_column("transaction_date", F.col("transaction_date").cast(T.DateType()))
    transactions.write.mode("overwrite").save_as_table("SALES")
    #transactions.write.csv('@CSV_STAGE/transactions.csv', overwrite=True, single=True)
    
   # Feedback data generation

    comments_df = session.table("comments_temp")
    
    feedback_df = session.sql(f"""
        SELECT 
        'FB-' || TO_CHAR(SEQ8()) AS feedback_id,
        'CUST-' || TO_CHAR(UNIFORM(1, 5000, RANDOM())) AS customer_id,
        DATEADD(DAY, -UNIFORM(0, 365, RANDOM()), CURRENT_DATE) AS chat_date,
        UNIFORM(1, 9, RANDOM()) AS internal_id
    FROM TABLE(GENERATOR(ROWCOUNT => {num_feedback_reports}))
    """)
    
    final_feedback_df = feedback_df.join(comments_df, feedback_df["internal_id"] == comments_df["id"]) \
                                    .select(feedback_df["feedback_id"], 
                                            feedback_df["customer_id"], 
                                            feedback_df["chat_date"], 
                                            feedback_df["internal_id"], 
                                            comments_df["comment"])
    
    final_feedback_df.write.mode("overwrite").save_as_table("FEEDBACK_RAW")
    #final_feedback_df.write.csv('@CSV_STAGE/final_feedback_df.csv', overwrite=True, single=True)


To simulate the data feeds, we are going to generate 4 months of data from 6 months since now

In [ ]:

num_days_per_batch = 30

first_timestamp = datetime.now().date() - timedelta(days=12* num_days_per_batch)

generate_data(first_timestamp, num_days_per_batch * 1)  

In [ ]:
select min(transaction_date), max(transaction_date) from sales;

Here we can use the power of LLMs to understand customer feedback and provide a sentiment score that we will be using later to determine probability of churn


In [ ]:
from snowflake.cortex import sentiment
import snowflake.snowpark.functions as F


feedback_raw_df = session.table("feedback_raw")

feedback_sentiment_df = feedback_raw_df.with_columns(["sentiment"], [sentiment(F.col("comment"))])

feedback_sentiment_df.write.mode("overwrite").save_as_table("feedback_sentiment")

### Feature Engineering

This function will create a profile for each customer who has already made a purchase in the shop. For a given timestamp, it will analyze the past transactions and feedback and using Snowflake analytical functions will generate features that will be used to traind and predict models.

We will store it as a Stored Procedure so it can be called from anywhere

In [ ]:
session.sql("CREATE stage IF NOT EXISTS ML_STAGE").collect()


In [ ]:
use schema dev;

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from snowflake.snowpark import types as T
from snowflake.snowpark import functions as F
from snowflake import snowpark

def uc01_feature_engineering(session: snowpark.Session, db: str, sc:str, cur_date: datetime, table_name: str):

    # In oder to label as churn or not, we have to take a look to the features churn_days before the day
    # we are doing the calculation. We calculater metrics based on that daytme and if there are no
    # transactions between the cur_date and that number of churn_days_number we label it as churn
    # we do this for having the real label. On new data, we just calculate the feature metrics
    
    # Load data
    customers_tbl = '.'.join([db, sc,'CUSTOMERS'])
    sales_tbl = '.'.join([db, sc,'SALES'])
    feedback_tbl = '.'.join([db, sc,'FEEDBACK_SENTIMENT'])
    
    customers_df = session.table(customers_tbl)
    sales_df = session.table(sales_tbl)
    sales_df_last_tran = session.table(sales_tbl)

    feedback_sentiment_df = session.table(feedback_tbl)    

    # we are only doing feature engineering for transactions before cur_date
    
    sales_df_last_tran = sales_df_last_tran.filter(F.col("transaction_date") < F.lit(cur_date))

    sales_df = sales_df.filter(F.col("transaction_date") < F.lit(cur_date ))
        
    # count only feedback before cur_date
    
    feedback_sentiment_df = feedback_sentiment_df.filter(F.col("chat_date") < F.lit(cur_date))
    
    sales_agg_df = (
        sales_df_last_tran.group_by("customer_id")
        .agg(
            F.max("transaction_date").alias("last_purchase_date"),
            F.sum("total_amount").alias("total_customer_value")
        )
    )
    
    def custom_column_naming(input_col, agg, window):
        return f"{agg}_{input_col}_{window.replace('-', 'past_')}"
                                                   
    sales_agg_orders_df = sales_df.analytics.time_series_agg(
            time_col="transaction_date",
            aggs={"total_amount": ["SUM", "COUNT"]},
            windows=["-7D","-1MM", "-2MM", "-3MM"],
            sliding_interval="1D",
            group_by=["CUSTOMER_ID"],
            col_formatter = custom_column_naming)

    sales_agg_last_purchase = sales_agg_df.join(
        sales_agg_orders_df,
        (sales_agg_df.last_purchase_date == sales_agg_orders_df.transaction_date) &
        (sales_agg_df.CUSTOMER_ID == sales_agg_orders_df.CUSTOMER_ID),
        "left").select(
            sales_agg_df["customer_id"].alias("CUSTOMER_ID"),
            sales_agg_df["total_customer_value"],
            sales_agg_df["last_purchase_date"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_orders_df["SUM_TOTAL_AMOUNT_PAST_3MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_orders_df["COUNT_TOTAL_AMOUNT_PAST_3MM"]
        )

    #  feedback data

    latest_feedback_df = (feedback_sentiment_df.group_by("customer_id")
            .agg(F.max("chat_date").alias("chat_date")))
    
    feedback_agg_df = feedback_sentiment_df.analytics.time_series_agg(
            time_col="chat_date",
            aggs={"sentiment": ["COUNT", "AVG", "MIN"]},
            windows=["-7D","-1MM", "-2MM", "-3MM"],
            sliding_interval="1D",
            group_by=["CUSTOMER_ID"],
            col_formatter = custom_column_naming)

    feedback_agg_latest_df = latest_feedback_df.join(
        feedback_agg_df, "chat_date", "left").select(
            latest_feedback_df["CUSTOMER_ID"].alias("CUSTOMER_ID"),
            feedback_agg_df["COUNT_SENTIMENT_PAST_7D"],
            feedback_agg_df["COUNT_SENTIMENT_PAST_1MM"],
            feedback_agg_df["COUNT_SENTIMENT_PAST_2MM"],
            feedback_agg_df["COUNT_SENTIMENT_PAST_3MM"],
            feedback_agg_df["AVG_SENTIMENT_PAST_7D"],
            feedback_agg_df["AVG_SENTIMENT_PAST_1MM"],
            feedback_agg_df["AVG_SENTIMENT_PAST_2MM"],
            feedback_agg_df["AVG_SENTIMENT_PAST_3MM"],
            feedback_agg_df["MIN_SENTIMENT_PAST_7D"],
            feedback_agg_df["MIN_SENTIMENT_PAST_1MM"],
            feedback_agg_df["MIN_SENTIMENT_PAST_2MM"],
            feedback_agg_df["MIN_SENTIMENT_PAST_3MM"],          
        )

    feedback_agg_latest_df.limit(10)
    
    # Join tables
    features_df = (
        customers_df.join(sales_agg_last_purchase, "customer_id", "left")
        .join(feedback_agg_latest_df, "customer_id", "left")
        .select(
            customers_df["customer_id"],
            customers_df["age"],
            customers_df["gender"],
            customers_df["location"],
            customers_df["customer_segment"],
            sales_agg_last_purchase["last_purchase_date"],
            feedback_agg_latest_df["COUNT_SENTIMENT_PAST_7D"],
            feedback_agg_latest_df["COUNT_SENTIMENT_PAST_1MM"],
            feedback_agg_latest_df["COUNT_SENTIMENT_PAST_2MM"],
            feedback_agg_latest_df["COUNT_SENTIMENT_PAST_3MM"],
            feedback_agg_latest_df["AVG_SENTIMENT_PAST_7D"],
            feedback_agg_latest_df["AVG_SENTIMENT_PAST_1MM"],
            feedback_agg_latest_df["AVG_SENTIMENT_PAST_2MM"],
            feedback_agg_latest_df["AVG_SENTIMENT_PAST_3MM"],
            feedback_agg_latest_df["MIN_SENTIMENT_PAST_7D"],
            feedback_agg_latest_df["MIN_SENTIMENT_PAST_1MM"],
            feedback_agg_latest_df["MIN_SENTIMENT_PAST_2MM"],
            feedback_agg_latest_df["MIN_SENTIMENT_PAST_3MM"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_7D"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_1MM"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_2MM"],
            sales_agg_last_purchase["SUM_TOTAL_AMOUNT_PAST_3MM"],
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_7D"].alias("COUNT_ORDERS_PAST_7D"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_1MM"].alias("COUNT_ORDERS_PAST_1MM"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_2MM"].alias("COUNT_ORDERS_PAST_2MM"),
            sales_agg_last_purchase["COUNT_TOTAL_AMOUNT_PAST_3MM"].alias("COUNT_ORDERS_PAST_3MM"),
            F.datediff("day", sales_agg_df["last_purchase_date"], F.lit(cur_date)).alias("DAYS_SINCE_LAST_PURCHASE"),
            F.lit(cur_date).alias("TIMESTAMP")
        ).filter(sales_agg_df["last_purchase_date"].isNotNull()  # Avoid customers never purchased
        ).dropDuplicates(["customer_id", "TIMESTAMP"])  # Ensure one combination of customer_id and TIMESTAMP

    )
    
    # Fill with 0 those where we have no data (so neutral feedback and zero iterations and amount)
    columns_to_fill = [
        "COUNT_SENTIMENT_PAST_7D", "COUNT_SENTIMENT_PAST_1MM", "COUNT_SENTIMENT_PAST_2MM", "COUNT_SENTIMENT_PAST_3MM",
        "AVG_SENTIMENT_PAST_7D", "AVG_SENTIMENT_PAST_1MM", "AVG_SENTIMENT_PAST_2MM", "AVG_SENTIMENT_PAST_3MM",
        "MIN_SENTIMENT_PAST_7D", "MIN_SENTIMENT_PAST_1MM", "MIN_SENTIMENT_PAST_2MM", "MIN_SENTIMENT_PAST_3MM",
        "SUM_TOTAL_AMOUNT_PAST_7D", "SUM_TOTAL_AMOUNT_PAST_1MM", "SUM_TOTAL_AMOUNT_PAST_2MM", "SUM_TOTAL_AMOUNT_PAST_3MM",
        "COUNT_ORDERS_PAST_7D", "COUNT_ORDERS_PAST_1MM", "COUNT_ORDERS_PAST_2MM", "COUNT_ORDERS_PAST_3MM"
    ]
    
    for column in columns_to_fill:
        features_df = features_df.fillna({column: 0})
    
    # Write to Snowflake Table
    features_df.write.mode("append").save_as_table(table_name)

    print (f'Created table {table_name}')

session.sproc.register(
    func=uc01_feature_engineering,
    name="uc01_feature_engineering_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)


In [ ]:
sales_df = session.table("sales")

first_sale_timestamp = sales_df.select(F.min(F.col("transaction_date"))).collect()[0][0]

last_sale_timestamp = sales_df.select(F.max(F.col("transaction_date"))).collect()[0][0]

print (f'First sale:{first_sale_timestamp}')
print (f'Last sale: {last_sale_timestamp}')

### Generate monthly purchase based on customer behaviors

In [ ]:
session.sql('use schema DEV;').collect()
print (session.get_current_schema())

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from snowflake.snowpark import types as T
from snowflake.snowpark import functions as F
from snowflake import snowpark

def add_new_top_sales (session: snowpark.Session, churn_base_table: str, 
                       new_sales_table:str , days_window: int):

    churn_df = session.table(churn_base_table)
    num_cust = churn_df.count()
        
    latest_timestamp = churn_df.select(F.max(F.col("TIMESTAMP"))).collect()[0][0]
    
    churn_df = churn_df.filter(F.col("TIMESTAMP") == F.lit(latest_timestamp))
    
    top_sentiment_customers_df = (
            churn_df.order_by(F.col("AVG_SENTIMENT_PAST_1MM").desc())).limit(round(num_cust*0.1))

    top_sentiment_customers_df = (
        churn_df
        .filter(F.col("AVG_SENTIMENT_PAST_1MM") > 0)  # Filter out to positive sentiment
        .orderBy(F.col("AVG_SENTIMENT_PAST_1MM").desc())  # Order by descending sentiment
        .limit(round(num_cust * 0.1))  # Select top 10% customers
    )
    
    top_sentiment_customers_df = top_sentiment_customers_df.select(F.col("customer_id")).collect()
    
    top_orders_customers_df = churn_df.order_by(F.col("COUNT_ORDERS_PAST_1MM").desc()).limit(round(num_cust*0.1))
    top_orders_customers_df = top_orders_customers_df.select(F.col("customer_id")).collect()

    #get a ramdom sample of customers to buy again
    sample_customers_buy_again = churn_df.select(F.col("customer_id"))\
            .sample(frac=0.1).collect()
    
    last_sales_timestamp = session.table("sales").select(F.max(F.col("transaction_date"))).collect()[0][0]
    
    def add_transactions (df, last_sales_timestamp, days_window):
    
        transactions = []
        
        for customer in df:

            for i in range(random.randint(1,5)): #between 1 and 10 transactions in the period
                customer_id = customer["CUSTOMER_ID"]
                    
                payment_methods = ['Credit Card', 'Paypal', 'Apple Pay']
            
                transaction_id = f"TRANS-N"
                customer_id = customer_id
                
                #if round(random.randint(1,10)) >= 6: # 60% buy in the entire period
                days=random.randint(1,days_window)
                #else: # the rest buy in the first quarter
                #    days=random.randint(1,round(days_window/4)+1)

                transaction_date = last_sales_timestamp + timedelta(days=random.randint(1,days))
                
                total_amount = round(random.uniform(50, 5000), 2)
                num_items = random.randint(1, 5)
                discount_applied = bool(random.randint(0, 1))
                payment_method = random.choice(payment_methods)
                    
                transactions.append([transaction_id, customer_id, transaction_date, total_amount, num_items, discount_applied, payment_method])
        
        # Creating the DataFrame for transactions
        df_transactions = pd.DataFrame(transactions, columns=[
            "TRANSACTION_ID", "CUSTOMER_ID", "TRANSACTION_DATE", "TOTAL_AMOUNT", 
            "NUM_ITEMS", "DISCOUNT_APPLIED", "PAYMENT_METHOD"
        ])
        
        df_transactions["TRANSACTION_DATE"] = pd.to_datetime(df_transactions["TRANSACTION_DATE"], errors="coerce")
    
        transactions = session.create_dataframe(df_transactions).drop_duplicates()
        transactions = transactions.with_column("TRANSACTION_DATE", F.to_date(F.col("TRANSACTION_DATE")))

       # transactions = transactions.with_column("transaction_date", F.col("transaction_date").cast(T.DateType()))
        transactions.write.mode("append").save_as_table(new_sales_table)
     
        num_tran = transactions.count()
        print (f'added {num_tran} transactions')

    print ("writing top sentiment custoemrs")
    add_transactions(top_sentiment_customers_df, last_sales_timestamp, days_window)

    print ("writing top orders custoemrs")
    add_transactions(top_orders_customers_df, last_sales_timestamp, days_window)

    print ("writing sample from all customers")
    add_transactions(sample_customers_buy_again, last_sales_timestamp, days_window)
    
    #angry customers
    angry_customers_df = (
            churn_df.order_by(F.col("AVG_SENTIMENT_PAST_1MM").asc())).limit(round(num_cust*0.05))

    angry_sentiment_customers_df = angry_customers_df.select(F.col("customer_id")).collect()

    #even some angry custoemrs buy again!
    angry_sample_buy_df = angry_customers_df\
            .select(F.col("customer_id"))\
            .sample(frac=0.1).collect()
 
    print ("Adding angry customers")
    add_transactions(angry_sample_buy_df, last_sales_timestamp, days_window)
 
    #add sentiment but no transactions (customer keep complaining)
    feedback = []

    for customer in angry_sentiment_customers_df:
        feedback_id = 'FEEDBACK-N'
        customer_id = customer["CUSTOMER_ID"]
        customer_id = customer_id
        chat_date = last_sales_timestamp + timedelta(days=random.randint(1,days_window))
        internal_id = random.randint(1, 3) #from angry to neutral
        feedback.append([feedback_id, customer_id, chat_date, internal_id])

    df_feedback = pd.DataFrame(feedback, columns=[
            "FEEDBACK_ID", "CUSTOMER_ID", "CHAT_DATE",
            "INTERNAL_ID"
    ])

    print ("writing feedback")
    feedback = session.create_dataframe(df_feedback).drop_duplicates()
    feedback = feedback.with_column("chat_date", F.col("chat_date").cast(T.DateType()))
    feedback = feedback.with_column("customer_id", F.col("customer_id").cast(T.StringType()))

    feedback.write.mode("overwrite").save_as_table("temp_feedback")

    feedback_df = session.table("temp_feedback")
    
    comments_df = session.table("comments_temp")
    
    temp_feedback_df = feedback_df.join(comments_df, feedback_df["internal_id"] == comments_df["id"]) \
                                    .select(feedback_df["feedback_id"], 
                                            feedback_df["customer_id"], 
                                            feedback_df["chat_date"], 
                                            feedback_df["internal_id"], 
                                            comments_df["comment"])
  
    feedback_sentiment_df = temp_feedback_df.with_columns(["sentiment"], [sentiment(F.col("comment"))])
  
    feedback_sentiment_df.write.mode("append").save_as_table('feedback_sentiment')

    return "Transactions added"

session.sproc.register(
    func=add_new_top_sales,
    name="add_new_top_sales_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)



In [ ]:
# Build features for the timestamp of the last sales. This will create the first customer 
# behavior profile in the training baseline baseline_features_builing_dataset
# With theat profile, we call the function to add more sales based on that customer profile

db = session.get_current_database()
sc = 'DEV'
table_features = 'baseline_features_builing_dataset'

session.sql(f'use schema {sc}').collect()
session.sql(f'drop table if exists {table_features}').collect()

sales_df = session.table("sales")
n_transactions = sales_df.count()
print (n_transactions)

feedback_sentiment_df = session.table("feedback_sentiment")
num_reviews = feedback_sentiment_df.count()
print (f'num reviews: {num_reviews}')

for i in range(12):

    # Customers profiles fora given timestmp (last sales date)
    sales_df = session.table("sales")

    last_sale_timestamp = sales_df.select(F.max(F.col("transaction_date"))).collect()[0][0]

    print (f'Building features for timestamp: {last_sale_timestamp}')
    uc01_feature_engineering (session, db, sc, last_sale_timestamp, table_features)

    #add 30 more days of sales to thee sales table, based on the last
    #profile of custoemr_churn_testing
    print (f'adding more sales')
    add_new_top_sales (session, table_features, 'sales',30)

    n_transactions = sales_df.count()
    print (n_transactions)
    

In [ ]:
select timestamp,  count(*) from baseline_features_builing_dataset
group by timestamp
order by timestamp desc;


In [ ]:
describe table baseline_features_builing_dataset;

In [ ]:
# Now that we have created the dataset, we need to add the churn label
    
def uc01_label_churn(session: snowpark.Session, baseline_table: str, output_table: str,  
                     num_days_churn: int):

    # Load baseline features dataset
    baseline_df = session.table(baseline_table)

    # Load sales dataset
    sales_df = session.table("SALES")

    # Filter sales to retain only customer ID and transaction date
    sales_filtered = sales_df.select(F.col("CUSTOMER_ID"), F.col("TRANSACTION_DATE"))

    # Find the next transaction date for each (CUSTOMER_ID, TIMESTAMP)
    next_transaction_df = (
        baseline_df
        .join(sales_filtered, "CUSTOMER_ID", "left")
        .filter(F.col("TRANSACTION_DATE") >F.col("LAST_PURCHASE_DATE"))
        .group_by(F.col("CUSTOMER_ID"), F.col("TIMESTAMP"))
        .agg(F.min("TRANSACTION_DATE").alias("NEXT_TRANSACTION_DATE"))
    )

    # Join back with the baseline dataset to compute CHURNED
    final_df = (
        baseline_df
        .join(next_transaction_df, ["CUSTOMER_ID", "TIMESTAMP"], "left")
        .select(
            baseline_df["*"],
            F.when(
                (F.col("NEXT_TRANSACTION_DATE").is_null()) |
                ((F.col("NEXT_TRANSACTION_DATE") - F.col("LAST_PURCHASE_DATE")) > num_days_churn),
                1
            ).otherwise(0).alias("CHURNED")
        )
    )

    # Save the final labeled dataset
    final_df.write.mode("overwrite").save_as_table(output_table)

session.sproc.register(
    func=uc01_label_churn,
    name="uc_01_label_churn_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=T.StringType()
)

In [ ]:
uc01_label_churn (session, 'baseline_features_builing_dataset', 'baseline_features_labeled', 30 )

In [ ]:
SELECT 
    TIMESTAMP,
    SUM(CASE WHEN churned = 0 THEN 1 ELSE 0 END) AS not_churned,
    SUM(CASE WHEN churned = 1 THEN 1 ELSE 0 END) AS churned
FROM baseline_features_labeled
GROUP BY TIMESTAMP
ORDER BY TIMESTAMP;

In [ ]:
import snowflake.snowpark as snowpark
import  snowflake.snowpark.functions as F
import datetime


def unload_table_by_week(session: snowpark.Session, stage_name: str, table: str, date_column: str):
    # Read sales data
    df = session.table(table)

    # Ensure transaction_date is in DATE format
    #df = df.with_column("transaction_date", to_date(col("transaction_date")))

    # Extract year and week for partitioning
    df = df.with_column("year", F.year(F.col(date_column)))
    df = df.with_column("week", F.weekofyear(F.col(date_column)))

    # Get distinct year-week pairs
    weeks = df.select("year", "week").distinct().collect()

    # Iterate over each week and export CSV
    for row in weeks:
        y, w = row["YEAR"], row["WEEK"]
        output_file = f"{table}_{y}_W{w}.csv"
        query = f"""
            COPY INTO @{stage_name}/{output_file}
            FROM (SELECT * FROM {table} WHERE year({date_column}) = {y} AND weekofyear({date_column}) = {w})
            FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY='"')
            SINGLE = TRUE;
        """
        session.sql(query).collect()
        print(f"Exported {output_file} to {stage_name}")

    return f"CSV export completed to {stage_name}"

In [ ]:
session.sql('use schema dev').collect()
session.sql("CREATE or replace stage CSV_STAGE").collect()
unload_table_by_week (session, 'CSV_STAGE', 'feedback_raw', 'chat_date')
unload_table_by_week (session, 'CSV_STAGE', 'sales', 'transaction_date')


In [ ]:
ls @CSV_STAGE;